# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Data loading functions. Uncomment the one you want to use
# Import other classifiers you plan to use
from worclipo.load_data import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier


data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')




The number of samples: 115
The number of columns: 494


In [8]:


def load_and_split_data(csv_path, target_col, test_size=0.2, random_state=42):
    """
    Load dataset from a CSV file and split into training and testing sets.
    
    Parameters:
    - csv_path: str, path to the CSV file.
    - target_col: str, name of the target variable column.
    - test_size: float, proportion of the dataset to include in the test split (default is 0.2).
    - random_state: int, controls the shuffling applied to the data before applying the split (default is 42).
    
    Returns:
    - X_train: DataFrame, training set features.
    - X_test: DataFrame, testing set features.
    - y_train: Series, training set target.
    - y_test: Series, testing set target.
    """
    
    # Load the dataset
    df = pd.read_csv(csv_path)
    
    # Separate features and target variable
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

csv_path = '/Users/anneloormemelink/Documents/Machine learning/TM10007_ML_Project_G17/worclipo/Lipo_radiomicFeatures.csv'  # Adjust this path to your CSV file's location
target_col = 'label'  # Replace 'Target' with the actual name of your target column
test_size = 0.2  # 20% of the data will be used for testing

# Call the function
X_train, X_test, y_train, y_test = load_and_split_data(csv_path, target_col, test_size)

# Optionally, print the sizes of the splits to verify
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


pd.DataFrame(X_train)



Training set size: (92, 494)
Testing set size: (23, 494)


,ID,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,...,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
31,Lipo-032_0,0.747468,0.202266,51.015128,7.956326,7.726307,6.415164,0.903337,0.121974,0.026353,...,0.000000,0.085219,3.264721,10.910715,0.0,0,0.353116,1378.152704,0.000000,14.423047
67,Lipo-068_0,0.544270,0.251173,101.266132,18.720028,16.281349,16.647647,0.781540,0.207953,0.038916,...,0.000000,0.131577,1.981239,3.470776,0.0,0,0.475538,26595.982858,0.107864,18.257968
93,Lipo-094_0,0.309913,0.298807,38.647617,12.223102,31.196946,31.390005,0.599946,0.240219,0.101089,...,0.087884,0.195699,0.971751,-0.110627,0.0,0,0.650749,6192.308599,0.311835,15.440969
109,Lipo-110_0,0.071433,0.072093,55.537675,28.977755,53.432402,37.531782,0.479446,0.189476,0.273625,...,0.000000,0.189907,1.492416,1.071069,0.0,0,0.647128,3857.539826,0.213592,14.584753
22,Lipo-023_0,0.112093,0.154467,114.502891,29.871527,72.621330,91.104487,0.335518,0.201027,0.111891,...,0.000000,0.120234,2.305696,4.973787,0.0,0,0.453355,22105.876703,0.052585,18.059540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Lipo-107_0,0.225764,0.227863,44.943215,12.428528,33.053023,31.805623,0.496125,0.269479,0.079050,...,0.017835,0.172631,1.249561,0.681224,0.0,0,0.597037,8647.265461,0.244726,16.224543
14,Lipo-015_0,0.220464,0.248484,121.617719,32.548892,31.792549,41.665958,0.443167,0.306294,0.083636,...,0.000000,0.143939,1.971421,3.354894,0.0,0,0.528672,20474.201992,0.121098,17.655626
92,Lipo-093_0,0.465954,0.126123,14.593406,5.142704,11.101785,4.618386,0.879122,0.090769,0.122727,...,0.000000,0.182382,1.086824,-0.123529,0.0,0,0.587261,253.140174,0.268553,10.927818
51,Lipo-052_0,0.843016,0.066672,42.923579,5.523957,7.441322,3.285970,0.966328,0.024588,0.018568,...,0.000000,0.126167,1.556051,2.061479,0.0,0,0.442900,3531.005489,0.163069,15.771713
